In [32]:
pip install ipynb_import_lib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ipynb_import_lib (from versions: none)
ERROR: No matching distribution found for ipynb_import_lib


In [31]:
import numpy as np
import ipynb_import_lib
ch07 = ipynb_import_lib.import_ipynb("./ch07.ipynb")
from dataset.mnist import load_mnist

ModuleNotFoundError: No module named 'ipynb_import_lib'

In [21]:
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        """Convolutionレイヤー

        Args:
            W (numpy.ndarray): フィルター（重み）、形状は(FN, C, FH, FW)。
            b (numpy.ndarray): バイアス、形状は(FN)。
            stride (int, optional): ストライド、デフォルトは1。
            pad (int, optional): パディング、デフォルトは0。
        """
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.dW = None      # 重みの微分値
        self.db = None      # バイアスの微分値

        self.x = None       # 逆伝播で必要になる、順伝播時の入力
        self.col_x = None   # 逆伝播で必要になる、順伝播時の入力のcol展開結果
        self.col_W = None   # 逆伝播で必要になる、順伝播時のフィルターのcol展開結果
    
    def forward(self,x):
        """順伝播

        Args:
            x (numpy.ndarray): 入力。形状は(N, C, H, W)。

        Returns:
            numpy.ndarray: 出力。形状は(N, FN, OH, OW)。
        """
        FN,C,FH,FW = self.W.shape  # FN:フィルター数、C:チャンネル数、FH:フィルターの高さ、FW:幅
        N,x_C,H,W = x.shape        # N:バッチサイズ、x_C:チャンネル数、H：入力データの高さ、W:幅
        # 自分で設定したエラーをはいて停止できる
        assert C == x_C, f'チャンネル数の不一致！[C]{C}, [x_C]{x_C}'

        # 出力のサイズ算出
        assert (H + 2 * self.pad - FH) % self.stride == 0, 'OHが割り切れない！'
        assert (W + 2 * self.pad - FW) % self.stride == 0, 'OWが割り切れない！'
        OH = int((H + 2 * self.pad - FH) / self.stride + 1)
        OW = int((W + 2 * self.pad - FW) / self.stride + 1)

        # 入力データを展開
        # (N, C, H, W) → (N * OH * OW, C * FH * FW)
        col_x = im2col(x, FH, FW, self.stride, self.pad)

        # フィルターを展開(.Tで転置)
        # (FN, C, FH, FW) → (FN, C * FH * FW)→(C * FH * FW, FN)
        col_W = self.W.reshape(FN, -1).T

        # 出力を算出（col_x, col_W, bに対する計算は、Affineレイヤーと全く同じ）
        # (N * OH * OW, C * FH * FW)・(C * FH * FW, FN) → (N * OH * OW, FN)
        out = np.dot(col_x, col_W) + self.b

        # 結果の整形
        # (N * OH * OW, FN) → (N, OH, OW, FN) → (N, FN, OH, OW)
        out = out.reshape(N, OH, OW, FN).transpose(0, 3, 1, 2)

        # 逆伝播のために保存
        self.x = x
        self.col_x = col_x
        self.col_W = col_W

        return out
    
    def backward(self, dout):
        """逆伝播

        Args:
            dout (numpy.ndarray): 右の層から伝わってくる微分値、形状は(N, FN, OH, OW)。

        Returns:
            numpy.ndarray: 微分値（勾配）、形状は(N, C, H, W)。
        """
        FN,C,FH,FW = self.W.shape  # 微分値の形状はWと同じ(FN, C, FH, FW)

        # 右の層からの微分値を展開
        # (N, FN, OH, OW) → (N, OH, OW, FN) → (N * OH * OW, FN)
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)

        # 微分値算出（col_x, col_W, bに対する計算は、Affineレイヤーと全く同じ）
        dcol_x = np.dot(dout, self.col_W.T)     # → (N * OH * OW, C * FH * FW)
        self.dW = np.dot(self.col_x.T, dout)    # → (C * FH * FW, FN)
        self.db = np.sum(dout, axis=0)          # → (FN)

        # フィルター（重み）の微分値の整形
        # (C * FH * FW, FN) → (FN, C * FH * FW) → (FN, C, FH, FW)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        # 結果（勾配）の整形
        # (N * OH * OW, C * FH * FW) → (N, C, H, W)
        dx = col2im(dcol_x, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

In [22]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        """Poolingレイヤー

        Args:
            pool_h (int): プーリング領域の高さ
            pool_w (int): プーリング領域の幅
            stride (int, optional): ストライド、デフォルトは1。
            pad (int, optional): パディング、デフォルトは0。
        """
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

        self.x = None           # 逆伝播で必要になる、順伝播時の入力
        self.arg_max = None     # 逆伝播で必要になる、順伝播時に採用したcol_x各行の位置

    def forward(self, x):
        """順伝播

        Args:
            x (numpy.ndarray): 入力、形状は(N, C, H, W)。

        Returns:
            numpy.ndarray: 出力、形状は(N, C, OH, OW)。
        """
        N, C, H, W = x.shape  # N:データ数、C:チャンネル数、H:高さ、W:幅

        # 出力のサイズ算出
        assert (H - self.pool_h) % self.stride == 0, 'OHが割り切れない！'
        assert (W - self.pool_w) % self.stride == 0, 'OWが割り切れない！'
        OH = int((H - self.pool_h) / self.stride + 1)
        OW = int((W - self.pool_w) / self.stride + 1)

        # 入力データを展開、整形
        # (N, C, H, W) → (N * OH * OW, C * PH * PW)
        col_x = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        # (N * OH * OW, C * PH * PW) → (N * OH * OW * C, PH * PW)
        col_x = col_x.reshape(-1, self.pool_h * self.pool_w)

        # 出力(最大値)を算出
        # (N * OH * OW * C, PH * PW) → (N * OH * OW * C)
        out = np.max(col_x, axis=1)

        # 出力の整形
        # (N * OH * OW * C) → (N, OH, OW, C) → (N, C, OH, OW)
        out = out.reshape(N, OH, OW, C).transpose(0, 3, 1, 2)

        # 逆伝播のために保存
        self.x = x
        self.arg_max = np.argmax(col_x, axis=1)  # col_x各行の最大値の位置（インデックス）

        return out

    def backward(self, dout):
        """逆伝播

        Args:
            dout (numpy.ndarray): 右の層から伝わってくる微分値、形状は(N, C, OH, OW)。

        Returns:
            numpy.ndarray: 微分値（勾配）、形状は(N, C, H, W)。
        """
        # 右の層からの微分値を整形
        # (N, C, OH, OW) → (N, OH, OW, C)
        dout = dout.transpose(0, 2, 3, 1)

        # 結果の微分値用のcolを0で初期化
        # (N * OH * OW * C, PH * PW)
        pool_size = self.pool_h * self.pool_w
        dcol_x = np.zeros((dout.size, pool_size))

        # 順伝播時に最大値として採用された位置にだけ、doutの微分値（＝doutまんま）をセット
        # 順伝播時に採用されなかった値の位置は初期化時の0のまま
        # （ReLUでxが0より大きい場合およびxが0以下の場合の処理と同じ）
        assert dout.size == self.arg_max.size, '順伝搬時のcol_xの行数と合わない'
        dcol_x[np.arange(self.arg_max.size), self.arg_max.flatten()] = \
            dout.flatten()

        # 結果の微分値の整形1
        # (N * OH * OW * C, PH * PW) → (N, OH, OW, C, PH * PW)
        dcol_x = dcol_x.reshape(dout.shape + (pool_size,))  # 最後の','は1要素のタプルを示す

        # 結果の微分値の整形2
        # (N, OH, OW, C, PH * PW) → (N * OH * OW, C * PH * PW)
        dcol_x = dcol_x.reshape(
            dcol_x.shape[0] * dcol_x.shape[1] * dcol_x.shape[2], -1
        )

        # 結果の微分値の整形3
        # (N * OH * OW, C * PH * PW) → (N, C, H, W)
        dx = col2im(
            dcol_x, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad
        )

        return dx

In [25]:
# ReLU
class ReLU:
    def __init__(self):
        """ReLUレイヤー
        """
        self.mask = None

    def forward(self, x):
        """順伝播

        Args:
            x (numpy.ndarray): 入力

        Returns:
            numpy.ndarray: 出力
        """
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        """逆伝播

        Args:
            dout (numpy.ndarray): 右の層から伝わってくる微分値

        Returns:
            numpy.ndarray: 微分値
        """
        dout[self.mask] = 0
        dx = dout

        return dx
    
# Affine
class Affine:

    def __init__(self, W, b):
        """Affineレイヤー

        Args:
            W (numpy.ndarray): 重み
            b (numpy.ndarray): バイアス
        """
        self.W = W                      # 重み
        self.b = b                      # バイアス
        self.x = None                   # 入力（2次元化後）
        self.dW = None                  # 重みの微分値
        self.db = None                  # バイアスの微分値
        self.original_x_shape = None    # 元の入力の形状（3次元以上の入力時用）

    def forward(self, x):
        """順伝播

        Args:
            x (numpy.ndarray): 入力

        Returns:
            numpy.ndarray: 出力
        """
        # 3次元以上（テンソル）の入力を2次元化
        self.original_x_shape = x.shape  # 形状を保存、逆伝播で戻す必要があるので
        x = x.reshape(x.shape[0], -1)
        self.x = x

        # 出力を算出
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        """逆伝播

        Args:
            dout (numpy.ndarray): 右の層から伝わってくる微分値

        Returns:
            numpy.ndarray: 微分値
        """
        # 微分値算出
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        # 元の形状に戻す
        dx = dx.reshape(*self.original_x_shape)
        return dx

# Softmax + 損失関数
class SoftmaxWithLoss:
    def __init__(self):
        """Softmax-with-Lossレイヤー
        """
        self.loss = None    # 損失
        self.y = None       # softmaxの出力
        self.t = None       # 教師データ（one-hot vector）

    def forward(self, x, t):
        """順伝播

        Args:
            x (numpy.ndarray): 入力
            t (numpy.ndarray): 教師データ

        Returns:
            float: 交差エントロピー誤差
        """
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        """逆伝播

        Args:
            dout (float, optional): 右の層から伝わってくる微分値。デフォルトは1。

        Returns:
            numpy.ndarray: 微分値
        """
        batch_size = self.t.shape[0]    # バッチの個数
        dx = (self.y - self.t) * (dout / batch_size)

        return dx

# ハイパーパラメータの更新
class AdaGrad:

    """AdaGrad"""

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
#           1e-7で0除算を避けている
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

class RMSprop:

    """RMSprop"""

    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] *= self.decay_rate
            self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

# Softmaxと交差エントロピー誤差メソッド
def softmax(x):
    """ソフトマックス関数

    Args:
        x (numpy.ndarray): 入力

    Returns:
        numpy.ndarray: 出力
    """
    # バッチ処理の場合xは(バッチの数, 10)の2次元配列になる。
    # この場合、ブロードキャストを使ってうまく画像ごとに計算する必要がある。
    # ここでは1次元でも2次元でも共通化できるようnp.max()やnp.sum()はaxis=-1で算出し、
    # そのままブロードキャストできるようkeepdims=Trueで次元を維持する。
    c = np.max(x, axis=-1, keepdims=True)
    exp_a = np.exp(x - c)  # オーバーフロー対策
    sum_exp_a = np.sum(exp_a, axis=-1, keepdims=True)
    y = exp_a / sum_exp_a
    return y


def cross_entropy_error(y, t):
    """交差エントロピー誤差の算出

    Args:
        y (numpy.ndarray): ニューラルネットワークの出力
        t (numpy.ndarray): 正解のラベル

    Returns:
        float: 交差エントロピー誤差
    """

    # データ1つ場合は形状を整形（1データ1行にする）
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 誤差を算出してバッチ数で正規化
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [1]:
# deep learningに必要なメソッドの準備
# coding: utf-8
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        """Dropoutレイヤー

        Args:
            dropout_ratio (float): 学習時のニューロンの消去割合、デフォルトは0.5。
        """
        self.dropout_ratio = dropout_ratio              # 学習時のニューロンの消去割合
        self.valid_ratio = 1.0 - self.dropout_ratio     # 学習時に生かしていた割合
        self.mask = None                                # 各ニューロンの消去有無を示すフラグの配列

    def forward(self, x, train_flg=True):
        """順伝播

        Args:
            x (numpy.ndarray): 入力
            train_flg (bool, optional): 学習中ならTrue、デフォルトはTrue。

        Returns:
            numpy.ndarray: 出力
        """
        if train_flg:
            # 学習時は消去するニューロンを決めるマスクを生成
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio

            # 出力を算出
            return x * self.mask

        else:
            # 認識時はニューロンは消去しないが、学習時の消去割合を加味した出力に調整する
            return x * self.valid_ratio

    def backward(self, dout):
        """逆伝播

        Args:
            dout (numpy.ndarray): 右の層から伝わってくる微分値

        Returns:
            numpy.ndarray: 微分値（勾配）
        """
        # 消去しなかったニューロンのみ右の層の微分値を逆伝播
        assert self.mask is not None, '順伝播なしに逆伝播が呼ばれた'
        return dout * self.mask

In [2]:
class Adam:

    def __init__(self, alpha=0.001, beta1=0.9, beta2=0.999):
        """Adamによるパラメーターの最適化

        Args:
            alpha (float, optional): 学習係数、デフォルトは0.001。
            beta1 (float, optional): Momentumにおける速度の過去と今の按分の係数、デフォルトは0.9。
            beta2 (float, optional): AdaGradにおける学習係数の過去と今の按分の係数、デフォルトは0.999。
        """
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2

        self.m = None   # Momentumにおける速度
        self.v = None   # AdaGradにおける学習係数
        self.t = 0      # タイムステップ

    def update(self, params, grads):
        """パラメーター更新

        Args:
            params (dict): 更新対象のパラメーターの辞書、keyは'W1'、'b1'など。
            grads (dict): paramsに対応する勾配の辞書
        """
        # mとvの初期化
        if self.m is None:
            self.m = {}
            self.v = {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)

        # 更新
        self.t += 1     # タイムステップ加算
        for key in params.keys():

            # mの更新、Momentumにおける速度の更新に相当
            # 過去と今の勾配を beta1 : 1 - beta1 で按分する
            self.m[key] = \
                self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]

            # vの更新、AdaGradにおける学習係数の更新に相当
            # 過去と今の勾配を beta2 : 1 - beta2 で按分する
            self.v[key] = \
                self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key] ** 2)

            # パラメーター更新のためのmとvの補正値算出
            hat_m = self.m[key] / (1.0 - self.beta1 ** self.t)
            hat_v = self.v[key] / (1.0 - self.beta2 ** self.t)

            # パラメーター更新、最後の1e-7は0除算回避
            params[key] -= self.alpha * hat_m / (np.sqrt(hat_v) + 1e-7)

In [3]:
def softmax(x):
    """ソフトマックス関数

    Args:
        x (numpy.ndarray): 入力

    Returns:
        numpy.ndarray: 出力
    """
    # バッチ処理の場合xは(バッチの数, 10)の2次元配列になる。
    # この場合、ブロードキャストを使ってうまく画像ごとに計算する必要がある。
    # ここでは1次元でも2次元でも共通化できるようnp.max()やnp.sum()はaxis=-1で算出し、
    # そのままブロードキャストできるようkeepdims=Trueで次元を維持する。
    c = np.max(x, axis=-1, keepdims=True)
    exp_a = np.exp(x - c)  # オーバーフロー対策
    sum_exp_a = np.sum(exp_a, axis=-1, keepdims=True)
    y = exp_a / sum_exp_a
    return y


def cross_entropy_error(y, t):
    """交差エントロピー誤差の算出

    Args:
        y (numpy.ndarray): ニューラルネットワークの出力
        t (numpy.ndarray): 正解のラベル

    Returns:
        float: 交差エントロピー誤差
    """

    # データ1つ場合は形状を整形（1データ1行にする）
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 誤差を算出してバッチ数で正規化
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size


def conv_output_size(input_size, filter_size, pad, stride):
    """畳み込み層の出力サイズ算出

    Args:
        input_size (int): 入力の1辺のサイズ（縦横は同値の前提）
        filter_size (int): フィルターの1辺のサイズ（縦横は同値の前提）
        pad (int): パディングのサイズ（縦横は同値の前提）
        stride (int): ストライド幅（縦横は同値の前提）

    Returns:
        int: 出力の1辺のサイズ
    """
    assert (input_size + 2 * pad - filter_size) \
        % stride == 0, '畳み込み層の出力サイズが割り切れない！'
    return int((input_size + 2 * pad - filter_size) / stride + 1)


def pool_output_size(input_size, pool_size, stride):
    """プーリング層の出力サイズ算出

    Args:
        input_size (int): 入力の1辺のサイズ（縦横は同値の前提）
        pool_size (int): プーリングのウインドウサイズ（縦横は同値の前提）
        stride (int): ストライド幅（縦横は同値の前提）

    Returns:
        int: 出力の1辺のサイズ
    """
    assert (input_size - pool_size) % stride == 0, 'プーリング層の出力サイズが割り切れない！'
    return int((input_size - pool_size) / stride + 1)

In [27]:
# im2colメソッドの中身→そういうものとして使う：順伝播
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング

    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

# 逆伝播の時に使う
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    col :
    input_shape : 入力データの形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad

    Returns
    -------

    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [8]:
# ディープなCNNの実装
# coding: utf-8
import numpy as np

class DeepConvNet:

    def __init__(
        self, input_dim=(1, 28, 28),
        conv_param_1={
            'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1
        },
        conv_param_2={
            'filter_num': 16, 'filter_size': 3, 'pad': 1, 'stride': 1
        },
        conv_param_3={
            'filter_num': 32, 'filter_size': 3, 'pad': 1, 'stride': 1
        },
        conv_param_4={
            'filter_num': 32, 'filter_size': 3, 'pad': 2, 'stride': 1
        },
        conv_param_5={
            'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1
        },
        conv_param_6={
            'filter_num': 64, 'filter_size': 3, 'pad': 1, 'stride': 1
        },
        hidden_size=50, output_size=10
    ):
        """ディープな畳み込みニューラルネットワーク

        Args:
            input_dim (tuple, optional): 入力データの形状、デフォルトは(1, 28, 28)。
            conv_param_1 (dict, optional): 畳み込み層1のハイパーパラメーター、
                デフォルトは{'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1}。
            conv_param_2 (dict, optional): 畳み込み層2のハイパーパラメーター、
                デフォルトは{'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1}。
            conv_param_3 (dict, optional): 畳み込み層3のハイパーパラメーター、
                デフォルトは{'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1}。
            conv_param_4 (dict, optional): 畳み込み層4のハイパーパラメーター、
                デフォルトは{'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1}。
            conv_param_5 (dict, optional): 畳み込み層5のハイパーパラメーター、
                デフォルトは{'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1}。
            conv_param_6 (dict, optional): 畳み込み層6のハイパーパラメーター、
                デフォルトは{'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1}。
            hidden_size (int, optional): 隠れ層のニューロンの数、デフォルトは50。
            output_size (int, optional): 出力層のニューロンの数、デフォルトは10。
        """
        assert input_dim[1] == input_dim[2], '入力データは高さと幅が同じ前提！'

        # パラメーターの初期化とレイヤー生成
        self.params = {}    # パラメーター
        self.layers = {}    # レイヤー（Python 3.7からは辞書の格納順が保持されるので、OrderedDictは不要）

        # 入力サイズ
        channel_num = input_dim[0]                          # 入力のチャンネル数
        input_size = input_dim[1]                           # 入力サイズ

        # [1] 畳み込み層#1 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_1.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W1', 'b1'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv1'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [2] ReLU層#1 : レイヤー生成
        self.layers['ReLU1'] = ReLU()

        # [3] 畳み込み層#2 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_2.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W2', 'b2'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv2'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [4] ReLU層#2 : レイヤー生成
        self.layers['ReLU2'] = ReLU()

        # [5] プーリング層#1 : レイヤー生成
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)

        # 次の層の入力サイズ算出
        input_size = pool_output_size(input_size, pool_size=2, stride=2)

        # [6] 畳み込み層#3 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_3.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W3', 'b3'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv3'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [7] ReLU層#3 : レイヤー生成
        self.layers['ReLU3'] = ReLU()

        # [8] 畳み込み層#4 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_4.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W4', 'b4'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv4'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [9] ReLU層#4 : レイヤー生成
        self.layers['ReLU4'] = ReLU()

        # [10] プーリング層#2 : レイヤー生成
        self.layers['Pool2'] = Pooling(pool_h=2, pool_w=2, stride=2)

        # 次の層の入力サイズ算出
        input_size = pool_output_size(input_size, pool_size=2, stride=2)

        # [11] 畳み込み層#5 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_5.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W5', 'b5'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv5'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [12] ReLU層#5 : レイヤー生成
        self.layers['ReLU5'] = ReLU()

        # [13] 畳み込み層#6 : パラメーター初期化とレイヤー生成
        filter_num, filter_size, pad, stride = list(conv_param_6.values())
        pre_node_num = channel_num * (filter_size ** 2)     # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W6', 'b6'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(filter_num, channel_num, filter_size, filter_size)
        )
        self.params[key_b] = np.zeros(filter_num)

        self.layers['Conv6'] = Convolution(
            self.params[key_w], self.params[key_b], stride, pad
        )

        # 次の層の入力サイズ算出
        channel_num = filter_num
        input_size = conv_output_size(input_size, filter_size, pad, stride)

        # [14] ReLU層#6 : レイヤー生成
        self.layers['ReLU6'] = ReLU()

        # [15] プーリング層#3 : レイヤー生成
        self.layers['Pool3'] = Pooling(pool_h=2, pool_w=2, stride=2)

        # 次の層の入力サイズ算出
        input_size = pool_output_size(input_size, pool_size=2, stride=2)

        # [16] Affine層#1　: パラメーター初期化とレイヤー生成
        pre_node_num = channel_num * (input_size ** 2)      # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W7', 'b7'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(channel_num * (input_size ** 2), hidden_size)
        )
        self.params[key_b] = np.zeros(hidden_size)

        self.layers['Affine1'] = Affine(self.params[key_w], self.params[key_b])

        # 次の層の入力サイズ算出
        input_size = hidden_size

        # [17] ReLU層#7 : レイヤー生成
        self.layers['ReLU7'] = ReLU()

        # [18] Dropout層#１ : レイヤー生成
        self.layers['Drop1'] = Dropout(dropout_ratio=0.5)

        # [19] Affine層#2　: パラメーター初期化とレイヤー生成
        pre_node_num = input_size                           # 1ノードに対する前層の接続ノード数
        key_w, key_b = 'W8', 'b8'                           # 辞書格納時のkey
        self.params[key_w] = np.random.normal(
            scale=np.sqrt(2.0 / pre_node_num),              # Heの初期値の標準偏差
            size=(input_size, output_size)
        )
        self.params[key_b] = np.zeros(output_size)

        self.layers['Affine2'] = Affine(self.params[key_w], self.params[key_b])

        # [20] Dropout層#2 : レイヤー生成
        self.layers['Drop2'] = Dropout(dropout_ratio=0.5)

        # [21] Softmax層 : レイヤー生成
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        """ニューラルネットワークによる推論

        Args:
            x (numpy.ndarray): ニューラルネットワークへの入力
            train_flg (Boolean): 学習中ならTrue（Dropout層でニューロンの消去を実施）

        Returns:
            numpy.ndarray: ニューラルネットワークの出力
        """
        # レイヤーを順伝播
        for layer in self.layers.values():
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)  # Dropout層の場合は、学習中かどうかを伝える
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        """損失関数の値算出

        Args:
            x (numpy.ndarray): ニューラルネットワークへの入力
            t (numpy.ndarray): 正解のラベル

        Returns:
            float: 損失関数の値
        """
        # 推論
        y = self.predict(x, True)   # 損失は学習中しか算出しないので常にTrue

        # Softmax-with-Lossレイヤーの順伝播で算出
        loss = self.lastLayer.forward(y, t)

        return loss

    def accuracy(self, x, t, batch_size=100):
        """認識精度算出
        batch_sizeは算出時のバッチサイズ。一度に大量データを算出しようとすると
        im2colでメモリを食い過ぎてスラッシングが起きてしまい動かなくなるため、
        その回避のためのもの。

        Args:
            x (numpy.ndarray): ニューラルネットワークへの入力
            t (numpy.ndarray): 正解のラベル（one-hot）
            batch_size (int), optional): 算出時のバッチサイズ、デフォルトは100。

        Returns:
            float: 認識精度
        """
        # 分割数算出
        batch_num = max(int(x.shape[0] / batch_size), 1)

        # 分割
        x_list = np.array_split(x, batch_num, 0)
        t_list = np.array_split(t, batch_num, 0)

        # 分割した単位で処理
        correct_num = 0  # 正答数の合計
        for (sub_x, sub_t) in zip(x_list, t_list):
            assert sub_x.shape[0] == sub_t.shape[0], '分割境界がずれた？'
            y = self.predict(sub_x, False)  # 認識精度は学習中は算出しないので常にFalse
            y = np.argmax(y, axis=1)
            t = np.argmax(sub_t, axis=1)
            correct_num += np.sum(y == t)

        # 認識精度の算出
        return correct_num / x.shape[0]

    def gradient(self, x, t):
        """重みパラメーターに対する勾配を誤差逆伝播法で算出

         Args:
            x (numpy.ndarray): ニューラルネットワークへの入力
            t (numpy.ndarray): 正解のラベル

        Returns:
            dictionary: 勾配を格納した辞書
        """
        # 順伝播
        self.loss(x, t)     # 損失値算出のために順伝播する

        # 逆伝播
        dout = self.lastLayer.backward()
        for layer in reversed(list(self.layers.values())):
            dout = layer.backward(dout)

        # 各レイヤーの微分値を取り出し
        grads = {}
        layer = self.layers['Conv1']
        grads['W1'], grads['b1'] = layer.dW, layer.db
        layer = self.layers['Conv2']
        grads['W2'], grads['b2'] = layer.dW, layer.db
        layer = self.layers['Conv3']
        grads['W3'], grads['b3'] = layer.dW, layer.db
        layer = self.layers['Conv4']
        grads['W4'], grads['b4'] = layer.dW, layer.db
        layer = self.layers['Conv5']
        grads['W5'], grads['b5'] = layer.dW, layer.db
        layer = self.layers['Conv6']
        grads['W6'], grads['b6'] = layer.dW, layer.db
        layer = self.layers['Affine1']
        grads['W7'], grads['b7'] = layer.dW, layer.db
        layer = self.layers['Affine2']
        grads['W8'], grads['b8'] = layer.dW, layer.db

        return grads

In [28]:
# MNISTの訓練データとテストデータ読み込み
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, flatten=False, one_hot_label=True)

# ハイパーパラメーター設定
iters_num = 12000           # 更新回数
batch_size = 100            # バッチサイズ
adam_param_alpha = 0.001    # Adamのパラメーター
adam_param_beta1 = 0.9      # Adamのパラメーター
adam_param_beta2 = 0.999    # Adamのパラメーター

train_size = x_train.shape[0]  # 訓練データのサイズ
iter_per_epoch = max(int(train_size / batch_size), 1)    # 1エポック当たりの繰り返し数

# ディープな畳み込みニューラルネットワーク生成
network = DeepConvNet()

# オプティマイザー生成、Adamを使用
optimizer = Adam(adam_param_alpha, adam_param_beta1, adam_param_beta2)

# 学習前の認識精度の確認
train_acc = network.accuracy(x_train, t_train)
test_acc = network.accuracy(x_test, t_test)
train_loss_list = []            # 損失関数の値の推移の格納先
train_acc_list = [train_acc]    # 訓練データに対する認識精度の推移の格納先
test_acc_list = [test_acc]      # テストデータに対する認識精度の推移の格納先
print(f'学習前 [訓練データの認識精度]{train_acc:.4f} [テストデータの認識精度]{test_acc:.4f}')

# 学習開始
for i in range(iters_num):

    # ミニバッチ生成
    batch_mask = np.random.choice(train_size, batch_size, replace=False)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 勾配の計算
    grads = network.gradient(x_batch, t_batch)

    # 重みパラメーター更新
    optimizer.update(network.params, grads)

    # 損失関数の値算出
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1エポックごとに認識精度算出
    if (i + 1) % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        # 経過表示
        print(
            f'[エポック]{(i + 1) // iter_per_epoch:>2} '
            f'[更新数]{i + 1:>5} [損失関数の値]{loss:.4f} '
            f'[訓練データの認識精度]{train_acc:.4f} [テストデータの認識精度]{test_acc:.4f}'
        )

# 損失関数の値の推移を描画
x = np.arange(len(train_loss_list))
plt.plot(x, train_loss_list, label='loss')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.xlim(left=0)
plt.ylim(0, 2.5)
plt.show()

# 訓練データとテストデータの認識精度の推移を描画
x2 = np.arange(len(train_acc_list))
plt.plot(x2, train_acc_list, label='train acc')
plt.plot(x2, test_acc_list, label='test acc', linestyle='--')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.xlim(left=0)
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

KeyboardInterrupt: 